# Describe a PIV Result File

In
this
example, we
want
to
describe
the
result
file
of
the
ILA
Vortex
pair(source: https: // www.pivtec.com / download / samples / VortexPairSeq.zip)

Where
to
start? Let
's first evaluate what we have:

General
context:
- The
general
concept
of
a
dataset is described
by
`dcat: Dataset
`
- The
file is described
by
`pivmeta: PIVResultDistribution
` and is part
of
the
`dcat: Dataset
`

Specific
information: < br >
Of
greater
interest is the
PIV
process
including
the
PIV
parameters
leading
to
the
dataset.A
`dcat: Dataset
` is the
output
of
the
PIV
process.For
this, we
can
use[`m4i:output
of
`](http: // purl.obolibrary.org / obo / RO_0002353).

## Preparation and Imports

In [ ]:
from ontolutils.ex import dcat, m4i, prov
from ssnolib import StandardNameTable

from pivmetalib import PIVMETA, pivmeta

# Describe a PIV Result File

In
this
example, we
want
to
describe
the
result
file
of
the
ILA
Vortex
pair(source: https: // www.pivtec.com / download / samples / VortexPairSeq.zip)

Where
to
start? Let
's first evaluate what we have:

General
context:
- The
general
concept
of
a
dataset is described
by
`dcat: Dataset
`
- The
file is described
by
`pivmeta: PIVResultDistribution
` and is part
of
the
`dcat: Dataset
`

Specific
information: < br >
Of
greater
interest is the
PIV
process
including
the
PIV
parameters
leading
to
the
dataset.A
`dcat: Dataset
` is the
output
of
the
PIV
process.For
this, we
can
use[`m4i:output
of
`](http: // purl.obolibrary.org / obo / RO_0002353).

## Preparation and Imports

In [ ]:
from ontolutils.ex import dcat, m4i, prov
from ssnolib import StandardNameTable
from ssnolib import StandardNameTable

from pivmetalib import PIVMETA, pivmeta

We will describe variables using standard names. For this we, decide to use [this Standard Name table](https://zenodo.org/records/15297431)

In [4]:
dist = dcat.Distribution(
    downloadURL='https://zenodo.org/records/17550472/files/Standard_Name_Table_for_Particle_Image_Velocimetry.jsonld',
    mediaType='application/json+ld'
)
dowloaded_filename = dist.download()

WindowsPath('C:/Users/matth/Documents/GitHub/pivmetalib/docs/Standard_Name_Table_for_Particle_Image_Velocimetry.jsonld')

In [5]:
dowloaded_filename

In [6]:
snt = StandardNameTable.parse(dowloaded_filename, fmt='jsonld')

In [7]:
standard_names = snt.get_standard_names_as_frozen_dataclass()

In [8]:
result_dist = pivmeta.ImageVelocimetryDistribution(
    title='Result File',
    downloadURL='file:///vp1a.dat',
    hasPIVDataType=PIVMETA.ResultData
)
# the "downloaded" file must exist:
assert result_dist.download().exists()

ImageVelocimetryDistribution(id=_:N792e543b2fd246b3be11a8d437598d4d, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)

In [9]:
piv_image_dist = pivmeta.ImageVelocimetryDistribution(
    title='ILA Vortex Pair Images',
    download_URL='https://www.pivtec.com/download/samples/pivimg1.zip',
    hasPIVDataType=PIVMETA.ExperimentalImage
)
piv_image_dist

Dataset(id=_:N117a52a2fec54c19a6a19c5184ce71dc, title=ILA Vortex Pair, distribution=[ImageVelocimetryDistribution(id=_:Nef4222e3694b470fa6fcd2da314702d4, title=Result File, downloadURL=file:///vp1a.dat, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ResultData), ImageVelocimetryDistribution(id=_:N792e543b2fd246b3be11a8d437598d4d, title=ILA Vortex Pair Images, downloadURL=https://www.pivtec.com/download/samples/pivimg1.zip, hasPIVDataType=https://matthiasprobst.github.io/pivmeta#ExperimentalImage)])

In [ ]:
ds = dcat.Dataset(
    title='ILA Vortex Pair',
    distribution=[result_dist, piv_image_dist]
)
ds

Before defining the PIV processing steps, we need to describe the software used:

In [ ]:
piv_software = pivmeta.PIVSoftware(
    author=prov.Organization(
        name='PIVTEC GmbH',
        mbox='info@pivtec.com',
        url='https://www.pivtec.com/'
    ),
    has_documentation='https://www.pivtec.com/download/docs/PIVview_v36_Manual.pdf',
)

## Processing steps

A `PIVProcessingStep` class is provided in order to distinguish the processing step from others. Some methods are provided as classes but without specific properties. This is done to provide flexibility, as all methods can be standardized. However, by introducing **standard names**, the authors may narrow their parameter definitions either by using global standard names (with an IRI) or within their project.

*TODO: Put here an image illustrating the possibilities*

By using standard names, important parameters can be identified unambiguously. See `PIVMETA.image_filter_kernel_size` in the example in contrast to the 180° image rotation.

### 1. Pre-Processing (Image processing)

Methods:
- image rotation by 180 deg

In [ ]:
pre = pivmeta.PIVPreProcessing(
    name='Image pre processing',
    realizesMethod=[
        m4i.Method(
            description='Rotates the input image by 180 deg',
            parameter=m4i.NumericalVariable(
                name='rotation',
                hasNumericalValue=180,
                unit='deg',
                quantity_kind='https://qudt.org/vocab/unit/DEG'
            )
        ),
        # Dont define all the classes for filters and outlier detection because everybody may define it differently.
        # common parameters can be specified "on demand" by standard names like so: 
        m4i.Method(
            description='Applies a median filter to the image',
            hasParameter=m4i.NumericalVariable(
                standard_name=standard_names.image_filter_kernel_size,
                hasNumericalValue=3,
            )
        )
    ]
)
print(pre.serialize("ttl"))

## 2. PIV evaluation

The PIV evaluation is a `m4i:ProcessingStep`. It realizes the methods
- `pivemta:CorrelationMethod` (e.g. FFT, ... ),
- `pivemta:InterrogationMethod` (e.g. `pivemta:Multigrid`, ...) and
- `pivemta:PeakSearchMethod`.

The outlier detection (validation) is a sub-processing step (`pivemta:PIVValidation`), because it can realize multiple methods. Most popular ones are described by classes:
- `pivemta:DynMean`.

Here, a multigrid evaluation is performed using standard FFT.

### 2.1 Correlation algorithm

The correlation algorithm is a subclass of `m4i:Method`. At least a name and description should be provided (here, taken from the documentation). We could also provide parameters.

In [ ]:
calgo = pivmeta.CorrelationMethod(
    name='Standard (FFT) Correlation',
    hasWindowWeightingFunction=PIVMETA.SquareWindow,
    description='The default mode of cross-correlation using FFTs to speed the computation. '
                'In principle the sum of pixel-wise multiplication of intensities is computed for each '
                'correlation offset (For implementation details please refer to Raffel et al. 2007).'
)
print(calgo.model_dump_ttl())

### 2.2 Interrogation method

In this example, a *Multi-Grid* method was used starting from a window with size 64 px down to 16 px in 3 steps

In [14]:
v = m4i.NumericalVariable(
    standard_name=standard_names.x_initial_interrogation_window_size,
    hasNumericalValue=64,
)
print(v.model_dump_ttl(resolve_keys=False))

In [15]:
#print(v.dump_jsonld())

NumericalVariable(id=_:N1e32b2afbb61496cb54d7da71551b219, description=Number of multigrid passes, hasNumericalValue=3)

In [ ]:
int_meth = m4i.Method(
    name='Multigrid interrogation method',
    description='Run a multigrid PIV algorithm on all images',
    hasParameter=[
        m4i.NumericalVariable(
            standard_name=standard_names.x_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_initial_interrogation_window_size,
            hasNumericalValue=64,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_size,
            hasNumericalValue=16,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.x_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            standard_name=standard_names.y_final_interrogation_window_overlap_size,
            hasNumericalValue=8,
        ),
        m4i.NumericalVariable(
            # hasStandardName=PIVMETA.number_of_multigrid_passes,
            description='Number of multigrid passes',
            hasNumericalValue=3,
        )
    ]
)
int_meth.hasParameter[-1]

### 2.3 Outlier detection

We use the following two methods for outlier detection:
- normalized median test threshold: 3.0 (see DOI=https://doi.org/10.1007/s00348-005-0016-6)
- dynamic mean test: mean=2.0, var=1.0

In [17]:
median_test = pivmeta.OutlierDetectionMethod(
    name='normalized median test',
    hasParameter=m4i.NumericalVariable(
        label='threshold',
        hasNumericalValue=3.0
    )
)

In [18]:
dyn_mean = pivmeta.OutlierDetectionMethod(
    name='dynamic mean test',
    hasParameter=[
        m4i.NumericalVariable(
            label='mean',
            hasNumericalValue=2.0
        ),
        m4i.NumericalVariable(
            label='var',
            hasNumericalValue=1.0
        )
    ]
)

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix m4i: <http://w3id.org/nfdi4ing/metadata4ing#> .
@prefix piv: <https://matthiasprobst.github.io/pivmeta#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .

[] a piv:PIVEvaluation ;
    rdfs:label "piv evaluation" ;
    m4i:realizesMethod [ a m4i:Method ;
            schema:description "Run a multigrid PIV algorithm on all images" ],
        [ a piv:OutlierDetectionMethod ],
        [ a piv:CorrelationMethod ;
            dcterms:description "The default mode of cross-correlation using FFTs to speed the computation. In principle the sum of pixel-wise multiplication of intensities is computed for each correlation offset (For implementation details please refer to Raffel et al. 2007)." ;
            piv:hasWindowWeightingFunction piv:SquareWindow ],
        [ a piv:OutlierDetectionMethod ] .




In [19]:
proc = pivmeta.PIVEvaluation(
    label='piv evaluation',
    realizesMethod=[
        calgo,
        int_meth,
        median_test,
        dyn_mean
    ]
)
print(proc.model_dump_ttl())

In [20]:
data_smoothing = m4i.Method(
    name='Low-pass filtering',
    description='applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise.',
    parameter=m4i.NumericalVariable(label='kernel', hasNumericalValue=2.0)
)

@prefix m4i: <http://w3id.org/nfdi4ing/metadata4ing#> .
@prefix piv: <https://matthiasprobst.github.io/pivmeta#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a piv:PIVProcessingStep ;
    rdfs:label "Post processing" ;
    m4i:realizesMethod [ a m4i:Method ;
            m4i:hasParameter [ a m4i:NumericalVariable ;
                    rdfs:label "kernel" ;
                    m4i:hasNumericalValue 2e+00 ] ;
            schema:description "applies a low-pass filtering on the data using a Gaussian weighted kernel of specified width to reduce spurious noise." ] .




In [ ]:
post = pivmeta.PIVPostProcessing(
    label='Post processing',
    realizesMethod=data_smoothing
)
print(post.model_dump_ttl())

## 3. Creating the full Meta document (connect information)

We created three processing steps:
1. pre (takes raw images)
2. proc
3. post (outputs result data)


In [ ]:
post.hasOutput = ds

## 4. dump PIV run to JSON-LD

In [23]:
piv = m4i.ProcessingStep(
    label='PIV Run',
    startsWith=pre,
    endsWith=post
)
# proc.part_of = piv
pre.precedes = proc
proc.precedes = post

# all processing steps were employed by software pivview:
pre.has_employed_tool = piv_software
proc.has_employed_tool = piv_software
post.has_employed_tool = piv_software

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix m4i: <http://w3id.org/nfdi4ing/metadata4ing#> .
@prefix piv: <https://matthiasprobst.github.io/pivmeta#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a piv:PIVPostProcessing ;
    m4i:precedes [ a piv:PIVEvaluation ;
            rdfs:label "piv evaluation" ;
            m4i:precedes [ a piv:PIVProcessingStep ;
                    rdfs:label "Post processing" ;
                    m4i:hasOutput [ a dcat:Dataset ;
                            dcterms:title "ILA Vortex Pair" ;
                            dcat:distribution [ a piv:ImageVelocimetryDistribution ;
                                    dcterms:title "ILA Vortex Pair Images" ;
                                    dcat:downloadURL <https://www.pivtec.com/download/samples/pivimg1.zip> ;
                                    piv:has

In [24]:
print(pre.model_dump_ttl())

In [ ]:
with open('piv_process.json', 'w') as f:
    f.write(proc.model_dump_jsonld())